In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 25.9 MB/s 
     |████████████████████████████████| 7.6 MB 46.4 MB/s 
     |████████████████████████████████| 182 kB 48.3 MB/s 


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext.data.metrics import bleu_score

from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

from tqdm.auto import tqdm

In [ ]:
data_file_open = open('/content/data_f.txt', 'r', encoding='UTF-8')
label_file_open = open('/content/label_f.txt', 'r', encoding='UTF-8')
data_lines=data_file_open.readlines()
label_lines = label_file_open.readlines()

input_dataset = []
output_dataset = []
dataset = []


for item in data_lines:
    input_dataset.append(item.strip())
for item in label_lines:
    output_dataset.append(item.strip())

In [4]:
print(len(input_dataset))
print(len(output_dataset))

42535
42535


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [6]:
dataset = []
for i in range(len(input_dataset)):
    data = input_dataset[i]+' = '+output_dataset[i]
    #if(len(data)>200):
    #    continue
    #if(len(data)<20):
    #    continue
    dataset.append(data)

In [7]:
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2)

In [8]:
print(len(train_dataset))
print(len(val_dataset))

34028
8507


In [9]:
tokenized_dataset_train = []

tokenized_dataset_val = []


for data in train_dataset:
    tokenized_dataset_train.append(tokenizer.encode(data))

for data in val_dataset:
    tokenized_dataset_val.append(tokenizer.encode(data))

In [10]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
training_args = TrainingArguments(
          output_dir='/content/train_1',
          per_device_train_batch_size=8,
          num_train_epochs=3
      )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [16]:
trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=tokenized_dataset_train
          val_dataset=tokenized_dataset_val)

In [17]:
trainer.train()

***** Running training *****
  Num examples = 34028
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12762
  Number of trainable parameters = 124439808


Step,Training Loss
500,3.215800
1000,3.113100
1500,3.064200
2000,2.997100
2500,2.994000
3000,2.974900
3500,2.923300
4000,2.919600
4500,2.795800
5000,2.678300


Saving model checkpoint to /content/train_1/checkpoint-500
Configuration saved in /content/train_1/checkpoint-500/config.json
Model weights saved in /content/train_1/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/train_1/checkpoint-1000
Configuration saved in /content/train_1/checkpoint-1000/config.json
Model weights saved in /content/train_1/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/train_1/checkpoint-1500
Configuration saved in /content/train_1/checkpoint-1500/config.json
Model weights saved in /content/train_1/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /content/train_1/checkpoint-2000
Configuration saved in /content/train_1/checkpoint-2000/config.json
Model weights saved in /content/train_1/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /content/train_1/checkpoint-2500
Configuration saved in /content/train_1/checkpoint-2500/config.json
Model weights saved in /content/train_1/checkpoint-2500/pytorch_mod

TrainOutput(global_step=12762, training_loss=2.7283679153306077, metrics={'train_runtime': 2255.7473, 'train_samples_per_second': 45.255, 'train_steps_per_second': 5.658, 'total_flos': 2708475632640000.0, 'train_loss': 2.7283679153306077, 'epoch': 3.0})

In [21]:
trainer.save_model()

Saving model checkpoint to /content/train_1
Configuration saved in /content/train_1/config.json
Model weights saved in /content/train_1/pytorch_model.bin


In [23]:
model = GPT2LMHeadModel.from_pretrained('/content/train_1')

loading configuration file /content/train_1/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_siz

In [93]:
train_dataset[20]

'I will always follow your instructions. = I am always bound to you.'

In [91]:
val_dataset[5]

'Oh, I will always be your protector, you fresh piece of meat. = O, my sweet beef, I must still be good angel to thee.'

In [24]:
input = tokenizer.encode(input_dataset[0]+' = ')

In [89]:
input = tokenizer.encode('Hello, how are you? = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

'Hello, how are you? = ? How now, how now! ? = How now, what now, ho! = How now, what now!  How now, how'

In [102]:
input = tokenizer.encode('You were either ignorant of it or, seeing it, why were you so childishly friendly? = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

'You were either ignorant of it or, seeing it, why were you so childishly friendly? =  O thou werest ignorant of it, nor, seeing it, why hast thou inest so unkind a nature! = O, nor else,'

In [101]:
input = tokenizer.encode('I will always follow your instructions. = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

'I will always follow your instructions. =  I will follow from thee every commandment. = Now I will follow thy counsel. = I will still follow thy office.    He that'

In [138]:
input = tokenizer.encode('We shall leave tomorrow. = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

"We shall leave tomorrow. =  Tomorrow shall we leave. = We shall away tomorrow. = Whoe'er we shall leave, tomorrow in all! = O, what further mischief"

In [139]:
model = GPT2LMHeadModel.from_pretrained('/content/')

loading configuration file /content/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 5025

OSError: ignored

In [140]:
from google.colab import drive

import shutil

drive.mount('/content/drive')

Mounted at /content/drive


In [143]:
shutil.copy("/content/train_1/config.json","/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_2/")
shutil.copy("/content/train_1/pytorch_model.bin","/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_2/")
shutil.copy("/content/train_1/training_args.bin","/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_2/")

'/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_1/training_args.bin'